In [1]:
# keras CNN predict air, between, ground or not jumping

In [2]:
!pip3 install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import pickle
from utils_misc import pickle_load_or_create

2024-06-24 13:16:29.659031: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-24 13:16:29.700007: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-24 13:16:30.453292: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
from DataGeneratorBordersDB import DataGeneratorSkillBorders

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
video_border_labels_path = 'df_video_border_labels_seq'
video_folder = '../videos/'

video_names = [
    '20240201_atelier_001.mp4',
    '20240201_atelier_002.mp4',
    '20240201_atelier_003.mp4',
    '20240201_atelier_004.mp4',
    '20240201_atelier_005.mp4',
    '20240209_atelier_006.mp4',
    '20240209_atelier_007.mp4',
    '20240209_atelier_008.mp4',
]

train_videos = [ video_folder + trainvid for trainvid in video_names]

In [6]:
config = pickle_load_or_create('convLSTM_20_3x3_rgb', lambda:{
    'convolution': (3,3),
    'time_length':16,
    'dim':64,
    'rgb':True
}, config=True)
df_labels = pickle_load_or_create(video_border_labels_path, lambda: [], config=False)
df_labels

,path,frame,border
0,20240201_atelier_001.mp4,0,9
1,20240201_atelier_001.mp4,1,9
2,20240201_atelier_001.mp4,2,9
3,20240201_atelier_001.mp4,3,9
4,20240201_atelier_001.mp4,4,9
...,...,...,...
2755,20240201_atelier_003.mp4,439,0
2756,20240201_atelier_003.mp4,440,1
2757,20240201_atelier_003.mp4,441,2
2758,20240201_atelier_003.mp4,442,2


In [7]:
df_labels.loc[df_labels.border == 5, 'border'] = 3
df_labels.loc[df_labels.border == 9, 'border'] = 4

In [8]:
def get_random_frame(videos, grey=True, scale=1):
    """
    videos: array of video_paths
    df_video_border_labels: panda dataframe ['path', 'frame', 'borderlabel']
          0 : ground
          1 : heels of ground
          2 : air
          3 : Fault
          4 : no skipper or not jumping
    """
    path = videos[np.random.randint(0, len(videos)-1)]
    
    cap = cv2.VideoCapture(path)
    video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    frame_nr = np.random.randint(0, video_length-1)
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_nr)
    res, frame = cap.read()
    frame = cv2.resize(frame, dsize=(0,0), fx=scale, fy=scale)
    # frame = cv2.cvtColor(frame, 7)
    cap.release()
    # cv2.destroyAllWindows()

    return path, frame_nr, frame

In [9]:
unique_labels = df_labels['border'].unique()
unique_labels

array([4, 0, 1, 2, 8])

In [10]:
input_shape = get_random_frame(train_videos, grey=True, scale=0.4)[2].shape
input_shape

(768, 432, 3)

In [11]:
config

{'convolution': (3, 3), 'time_length': 16, 'dim': 64, 'rgb': True}

In [12]:
convolution = config['convolution']

In [13]:
convolution

(3, 3)

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, TimeDistributed, Dense, Flatten
from tensorflow.keras.optimizers import Adam

model = Sequential()

model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3),
                     input_shape=(20, 64, 64, 3),
                     padding='same', return_sequences=True))
model.add(BatchNormalization())

model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3),
                     padding='same', return_sequences=True))
model.add(BatchNormalization())

# Uncomment if you want to add more ConvLSTM2D layers
# model.add(ConvLSTM2D(filters=64, kernel_size=(3, 3),
#                      padding='same', return_sequences=True))
# model.add(BatchNormalization())

# Apply TimeDistributed Dense layer to each time step
model.add(TimeDistributed(Flatten()))  # Flatten each frame
model.add(TimeDistributed(Dense(128, activation='relu')))
model.add(TimeDistributed(Dense(len(unique_labels), activation='softmax')))

model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


/home/miked/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_lstm2d (ConvLSTM2D)        │ (None, 20, 64, 64, 64) │       154,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 20, 64, 64, 64) │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_1 (ConvLSTM2D)      │ (None, 20, 64, 64, 64) │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 20, 64, 64, 64) │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 20, 262144)     │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 20, 128)        │    33,554,560 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 20, 5)          │           645 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,005,509 (129.72 MB)

 Trainable params: 34,005,253 (129.72 MB)

 Non-trainable params: 256 (1.00 KB)

In [16]:
# Parameters
params = {'dim': (64,64),
          'time_length':16,
          'batch_size': 20,
          'n_classes': len(unique_labels),
          'n_channels': 3,
          'shuffle': True,
}

training_generator = DataGeneratorSkillBorders(train=True, **params)
test_generator = DataGeneratorSkillBorders(train=False, **params)

connection established
DataGeneratorSkillBorders init done
on_epoch_end_called
   videoID  batches_in_video  random_order  total_batches
0        3                23             1             23
1        1                24             2             47
2        6                16             3             63
3        2                38             4            101
4        4                19             5            120
connection established
DataGeneratorSkillBorders init done
on_epoch_end_called
   videoID  batches_in_video  random_order  total_batches
0        5                20             1             20


In [17]:
training_generator.batch_order

,videoID,batches_in_video,random_order,total_batches
0,3,23,1,23
1,1,24,2,47
2,6,16,3,63
3,2,38,4,101
4,4,19,5,120


In [18]:
%%time
X, y = training_generator.__getitem__(25)

 getitem, batch_nr 25 train is: True
CPU times: user 2.01 s, sys: 66 ms, total: 2.08 s
Wall time: 442 ms


In [19]:
X.shape

(1, 20, 64, 64, 3)

In [20]:
X

array([[[[[0.31764706, 0.34901961, 0.36470588],
          [0.31764706, 0.35294118, 0.35686275],
          [0.32941176, 0.36862745, 0.36470588],
          ...,
          [0.74509804, 0.76470588, 0.76470588],
          [0.70980392, 0.74117647, 0.7372549 ],
          [0.42352941, 0.45490196, 0.45098039]],

         [[0.28235294, 0.31764706, 0.32156863],
          [0.3372549 , 0.37254902, 0.37647059],
          [0.23137255, 0.25882353, 0.2627451 ],
          ...,
          [0.70196078, 0.71372549, 0.71372549],
          [0.65098039, 0.65882353, 0.66666667],
          [0.65882353, 0.67058824, 0.67058824]],

         [[0.49019608, 0.54509804, 0.56470588],
          [0.53333333, 0.58431373, 0.59607843],
          [0.08627451, 0.11764706, 0.11372549],
          ...,
          [0.61176471, 0.61176471, 0.61176471],
          [0.61568627, 0.61568627, 0.61568627],
          [0.6       , 0.61176471, 0.60392157]],

         ...,

         [[0.39607843, 0.38039216, 0.36470588],
          [0.38039216,

In [21]:
y.shape

(1, 20, 1)

In [22]:
y

array([[[3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [0],
        [0],
        [0],
        [0]]])

In [23]:
y.shape


(1, 20, 1)

In [24]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

history = model.fit(training_generator, epochs=2, 
                    validation_data=test_generator, shuffle=False)

len called len is: 120
len called len is: 120
 getitem, batch_nr 0 train is: True
 getitem, batch_nr 1 train is: True
Epoch 1/2
len called len is: 120
len called len is: 120
 getitem, batch_nr 0 train is: True
 getitem, batch_nr 1 train is: True
  1/120 ━━━━━━━━━━━━━━━━━━━━ 16:52 9s/step - accuracy: 0.0000e+00 - loss: 2.8266 getitem, batch_nr 2 train is: True
  2/120 ━━━━━━━━━━━━━━━━━━━━ 5:38 3s/step - accuracy: 0.2500 - loss: 2.1200      getitem, batch_nr 3 train is: True
  3/120 ━━━━━━━━━━━━━━━━━━━━ 5:41 3s/step - accuracy: 0.3889 - loss: 1.7274 getitem, batch_nr 4 train is: True
  4/120 ━━━━━━━━━━━━━━━━━━━━ 5:45 3s/step - accuracy: 0.4760 - loss: 1.4805 getitem, batch_nr 5 train is: True
  5/120 ━━━━━━━━━━━━━━━━━━━━ 5:45 3s/step - accuracy: 0.5388 - loss: 1.3028 getitem, batch_nr 6 train is: True
  6/120 ━━━━━━━━━━━━━━━━━━━━ 5:43 3s/step - accuracy: 0.5810 - loss: 1.2574 getitem, batch_nr 7 train is: True
  7/120 ━━━━━━━━━━━━━━━━━━━━ 5:40 3s/step - accuracy: 0.5949 - loss: 1.5329 ge

In [25]:
pd.DataFrame(history.history)

,accuracy,loss,val_accuracy,val_loss
0,0.355000,10.262375,0.4150,9.429147
1,0.400417,9.660134,0.4075,9.549971


In [26]:
model.fit?

Signature:
model.fit(
    x=None,
    y=None,
    batch_size=None,
    epochs=1,
    verbose='auto',
    callbacks=None,
    validation_split=0.0,
    validation_data=None,
    shuffle=True,
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
)
Docstring:
Trains the model for a fixed number of epochs (dataset iterations).

Args:
    x: Input data. It could be:
        - A NumPy array (or array-like), or a list of arrays
        (in case the model has multiple inputs).
        - A tensor, or a list of tensors
        (in case the model has multiple inputs).
        - A dict mapping input names to the corresponding array/tensors,
        if the model has named inputs.
        - A `tf.data.Dataset`. Should return a tuple
        of either `(inputs, targets)` or
        `(inputs, targets, sample_weights)`.
        - A `keras.utils.PyDataset` returning `(inputs,
      

In [27]:
with open('../models/last_cnn_model_history.pkl', 'wb') as handle:
    pickle.dump(history, handle)